# 1. Loading and organizing data

In [7]:
# loading datasets
import pandas as pd

# Vertebral Column
# dataset for classification between Normal (NO) and Abnormal (AB)
vc2c = pd.read_csv('vertebral_column_data/column_2C.dat', delim_whitespace=True, header=None)
# dataset for classification between DH (Disk Hernia), Spondylolisthesis (SL) and Normal (NO)
vc3c = pd.read_csv('vertebral_column_data/column_3C.dat', delim_whitespace=True, header=None)

# Wall-Following
# dataset with all 24 ultrassound sensors readings
wf24f = pd.read_csv('wall_following_data/sensor_readings_24.data', header=None)
# dataset with simplified 4 readings (front, left, right and back)
wf4f  = pd.read_csv('wall_following_data/sensor_readings_4.data',  header=None)
# dataset with simplified 2 readings (front and left)
wf2f  = pd.read_csv('wall_following_data/sensor_readings_2.data',  header=None)

# Parkinson (31 people, 23 with Parkinson's disease (PD))
temp = pd.read_csv('parkinson_data/parkinsons.data')
labels = temp.columns.values.tolist()
new_labels = [label for label in labels if label not in ('name')] # taking off column 'name'
pk = temp[new_labels]

In [8]:
pk_features = pk.columns.tolist()
pk_features.remove('status')

# datasets with separation between 'features' and 'labels'
datasets = {
    "vc2c":  {"features": vc2c.iloc[:,0:6],  "labels": pd.get_dummies(vc2c.iloc[:,6],  drop_first=True)},
    "vc3c":  {"features": vc3c.iloc[:,0:6],  "labels": pd.get_dummies(vc3c.iloc[:,6],  drop_first=True)},
    "wf24f": {"features": wf24f.iloc[:,0:24],"labels": pd.get_dummies(wf24f.iloc[:,24],drop_first=True)},
    "wf4f":  {"features": wf4f.iloc[:,0:4],  "labels": pd.get_dummies(wf4f.iloc[:,4],  drop_first=True)},
    "wf2f":  {"features": wf2f.iloc[:,0:2],  "labels": pd.get_dummies(wf2f.iloc[:,2],  drop_first=True)},
    "pk":    {"features": pk.loc[:,pk_features], "labels": pk.loc[:,["status"]]}
}

In [9]:
for dataset in datasets:
    print("Dataset name: {}\nNumber of features: {}\nNumber of samples: {}\n".format(
        dataset, datasets[dataset]["features"].shape[1], datasets[dataset]["features"].shape[0]
    ))

Dataset name: vc2c
Number of features: 6
Number of samples: 310

Dataset name: vc3c
Number of features: 6
Number of samples: 310

Dataset name: wf24f
Number of features: 24
Number of samples: 5456

Dataset name: wf4f
Number of features: 4
Number of samples: 5456

Dataset name: wf2f
Number of features: 2
Number of samples: 5456

Dataset name: pk
Number of features: 22
Number of samples: 195



In [10]:
# Auxiliary function to tell when processing is over
def is_over(): # linux os
    import os
    os.system('spd-say "your program has finished"')

# 2. SOM studies

In [11]:
# importing old som
%run -i 'som.py'

In [16]:
import cupy as cp

In [17]:
# New SOM
import numpy as np
import plotly.offline as plt
import plotly.graph_objs as go
from random import randint
import ipywidgets as widgets
from IPython.display import clear_output
from plotly import tools
from multiprocessing import Pool

plt.init_notebook_mode(connected=True) # enabling plotly inside jupyter notebook

class SOM:
    'Class of Self Organizing Maps conected in a two-dimensional grid.'
    
    def __init__(self, nRows, nColumns): 
        self.nRows    = nRows
        self.nColumns = nColumns
        self.__epochs = 0 # number of epochs of trained SOM
        
        self.neurons = None
        self.indexes = None # list of index2D <=> index1D
        
        self.neuronsHist = None
        self.ssdHist     = None
        
    def init(self, X): # giving the data, so we can define maximum and minimum in each dimension
        # reset neuronsHist and ssdHist
        self.neuronsHist = None 
        self.ssdHist     = None
        
        dim = X.shape[1] # number of features
        rand = np.random.rand(self.nRows*self.nColumns, dim) # Auxiliary random element
        # find min-max for each dimension
        minimum = np.amin(X, axis=0)
        maximum = np.amax(X, axis=0)
        # Initializing neurons in random positions between minimum and maximum of the dataset
        self.neurons = (maximum - minimum)*rand + minimum
        
        # list of index2D == index1D
        self.indexes = self.index1Dto2D(np.arange(len(self.neurons))) 
        
    
    def fit(self, X, alpha0, sigma0, nEpochs=100, saveNeuronsHist=False, saveSSDHist=True, tol=1e-6,
              verboses=0, batchSize=np.inf):
        if self.neurons is None: self.init(X) # if self.init() wasn't run
        
        tau1 = nEpochs/sigma0
        tau2 = nEpochs
        SSD_new = self.SSD(X) # initial SSD, from random parameters
        
        if saveNeuronsHist: 
            self.neuronsHist    = [np.zeros(self.neurons.shape)]*(nEpochs+1)
            self.neuronsHist[0] = np.copy(self.neurons) # random neurons
        if saveSSDHist:
            self.ssdHist    = np.zeros((nEpochs+1))
            self.ssdHist[0] = SSD_new # initial SSD, from random parameters
        
        sigma = sigma0
        alpha = alpha0
        inertia = np.inf # initial value of inertia
        batchSize = min(len(X), batchSize) # adjusting ill defined batchSize
        for epoch in range(nEpochs):
            # Updating alpha and sigma
            sigma = sigma0*np.exp(-epoch/tau1);
            alpha = alpha0*np.exp(-epoch/tau2);
            
            order = np.random.permutation(len(X)) # shuffling order
            for i in order[:batchSize]: # for each datapoint in the shuflled order until batchSize
                # search for winner neuron
                winner_idx = self.get_winner(X[i])
                # neighbor function
                h_ik = self.h_neighbor(winner_idx, self.indexes, sigma)
                # updating neurons
                self.neurons += (h_ik[:, np.newaxis]*(X[i] - self.neurons))*alpha
                
            
            self.__epochs+=1 # updating number of epochs trained
            if verboses==1: print("End of epoch {}".format(epoch+1))
            
            SSD_old = SSD_new
            SSD_new = self.SSD(X)
            inertia = abs((SSD_old - SSD_new)/SSD_old)
            
            # Saving if necessary
            if saveNeuronsHist: self.neuronsHist[epoch+1] = np.copy(self.neurons)
            if saveSSDHist:     self.ssdHist[epoch+1]     = SSD_new
                       
            # breaking if tolerance reached before nEpochs
            if inertia < tol:
                # history cutting
                if saveNeuronsHist: self.neuronsHist = self.neuronsHist[0:epoch+2]
                if saveSSDHist:     self.ssdHist     = self.ssdHist[0:epoch+2]
                break
            
            
    def SSD(self, X):
        SSD = 0
        for x in X: # can it be vectorized?
            dist_2 = np.sum((self.neurons - x)**2, axis=1) # norm**2
            SSD += np.amin(dist_2)
        return SSD
        
    def index1Dto2D(self, index): # convert index of neurons parameter matrix to the 2D grid index
        return np.asarray([np.ceil((index+1)/self.nRows)-1, index%self.nRows], dtype='int').T
    
    def get_winner(self, x, dim=2):
        dist_2 = np.sum((self.neurons - x)**2, axis=1) # norm**2
        temp = np.argmin(dist_2)
        winner = self.index1Dto2D(temp) if dim==2 else temp
        return winner
        
    def h_neighbor(self, idx_1, idx_2, sigma):
        dist_2 = np.sum((idx_2 - idx_1)**2, axis=1) # norm**2
        return np.exp( -dist_2/(2*sigma**2) )
    
    def getLabels(self, X, dim=1): # get labels in 1 dimension or 2 dimension (neuron grid)
        N = len(X)
        labels = np.zeros((N,dim), dtype='int')
        for i in range(N):
            labels[i,:] = self.get_winner(X[i,:],dim)
        return labels
    
    def plotSSD(self):
        traceData = go.Scatter(
            x = [i+1 for i in range(self.__epochs)], # epochs
            y = self.ssdHist, 
            mode='lines',
            name='SSD')
        data = [traceData]
        layoutData = go.Layout(
            title = "SSD history",
            xaxis=dict(title='Epoch'),
            yaxis=dict(title='SSD')
        )

        fig = go.Figure(data=data, layout=layoutData)
        plt.iplot(fig)
    
        
    def plotSOM(self, X=None):
        if self.neuronsHist is not None:
            # Int box to change the iteration number
            n_txt = widgets.BoundedIntText(
                value=0,
                min=0,
                max=len(self.neuronsHist)-1,
                step=10,
                description='epoch:'
            )    
        ###############################################################
        # Function to draw the graph
        def upgradeChart(change):
            clear_output()
            if self.neuronsHist is not None:
                display(n_txt)    
                n_ = change['new'] # new iteration number

            if self.neuronsHist is not None:
                x = self.neuronsHist[n_][:,0].tolist() 
                y = self.neuronsHist[n_][:,1].tolist()
                name = 'neurons [epoch ='+str(n_)+']'
            else:
                x = self.neurons[:,0].tolist()
                y = self.neurons[:,1].tolist()
                name = 'neurons'

            neurons = go.Scatter(x=x, y=y, mode='markers', name=name, 
                                 marker = dict(size=10,color = '#673AB7'))

            # cada linha que conecta os neurônios
            linhas = [{}]*(2*self.nRows*self.nColumns - self.nRows - self.nColumns)
            count=0 #contador para saber qual linha estamos
            for linha in range(self.nRows): # conecta da esquerda para direita
                for coluna in range(self.nColumns): # e de cima para baixo
                    try:
                        if self.neuronsHist is not None:
                            x0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna+1)).all(axis=1))[0][0], 0]
                            y1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna+1)).all(axis=1))[0][0], 1]
                        else:
                            x0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neurons[np.where((self.indexes==(linha,coluna+1)).all(axis=1))[0][0], 0]
                            y1 = self.neurons[np.where((self.indexes==(linha,coluna+1)).all(axis=1))[0][0], 1]

                        linhas[count]= {'type':'line','x0':x0,'y0': y0,'x1':x1,'y1':y1,
                                        'line': {'color': '#673AB7','width': 1,}}
                        count+=1
                    except: # edge of the grid
                        pass
                    try:
                        if self.neuronsHist is not None:
                            x0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha+1, coluna)).all(axis=1))[0][0], 0]
                            y1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha+1, coluna)).all(axis=1))[0][0], 1]
                        else:
                            x0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neurons[np.where((self.indexes==(linha+1,coluna)).all(axis=1))[0][0], 0]
                            y1 = self.neurons[np.where((self.indexes==(linha+1,coluna)).all(axis=1))[0][0], 1]

                        linhas[count] = {'type': 'line','x0': x0,'y0': y0,'x1': x1,'y1': y1,
                                         'line': {'color': '#673AB7','width': 1}}
                        count+=1
                    except: # edge of the grid
                        pass

            data = []
            title = ""
            if X is not None:
                datapoints = go.Scatter(x = X[:,0], y = X[:,1], mode='markers', name='data',
                                        marker = dict(size = 5,color = '#03A9F4'))
                data = [datapoints, neurons]
                title = "Data + SOM"
            else:
                data = [neurons]
                title = "SOM"

            layout = go.Layout(title=title, xaxis=dict(title="$x_1$"), yaxis=dict(title="$x_2$"),
                               shapes=linhas)

            fig = go.Figure(data=data, layout=layout)
            plt.iplot(fig)
        ###########################################################################

        if self.neuronsHist is not None: n_txt.observe(upgradeChart, names='value')
        upgradeChart({'new': 0})

# Testing consistency

In [13]:
import cupy as cp
x = cp.arange(6).reshape(2, 3).astype('f')
print(x)
print(x.sum(axis=1))

[[0. 1. 2.]
 [3. 4. 5.]]
[ 3. 12.]


### SSD curves for parkinson dataset:

In [14]:
%%time
from math import ceil
# choosing and preparing dataset

dataset_name='pk'

data = datasets[dataset_name]
N = len(data['features'].index) # number of datapoints
l = ceil((5*N**.5)**.5) # tamanho do lado da grid quadrada de neurônios
X = data['features'].values.copy()
X1, X2 = scale_feat(X,X,scaleType='min-max')
X=X1

params = {
     "X":         X
    ,"alpha0":    0.1
    ,"sigma0":    3 
    ,"nEpochs":   50
    ,"verboses":  0
    #,"saveNeuronsHist": True
}

som_old = SOM_2D(l, l, X.shape[1]); som_old.init(X)
som_new = SOM(l, l)

som_new.fit(**params)
som_old.train(**params, batchSize=len(X))

print("SOM_new:"); som_new.plotSSD()
print("SOM_old:"); som_old.plotSSD()
print("SOM_old last SSD: {}\nSOM_new last SSD: {}".format(som_old.ssdHist[-1], som_new.ssdHist[-1]))

TypeError: _amin() got an unexpected keyword argument 'dtype'

### SSD curves and neurons evolution for Wall-Following data set:

In [10]:
%%time
from math import ceil
import datetime
# choosing and preparing dataset

dataset_name='wf2f'

data = datasets[dataset_name]
N = len(data['features'].index) # number of datapoints
l = ceil((5*N**.5)**.5) # tamanho do lado da grid quadrada de neurônios
X = data['features'].values.copy()
X1, X2 = scale_feat(X,X,scaleType='min-max')
X=X1

params = {
     "X":         X
    ,"alpha0":    0.1
    ,"sigma0":    3 
    ,"nEpochs":   100
    ,"verboses":  1
    #,"saveNeuronsHist": True
}

som_old = SOM_2D(l, l, X.shape[1]); som_old.init(X)
som_new = SOM(l, l)

print("SOM_old training started at {}".format(datetime.datetime.now()))
som_old.train(**params, batchSize=len(X), saveParam=True)
print("SOM_new training started at {}".format(datetime.datetime.now()))
som_new.fit(**params,saveNeuronsHist=True)
# nEpochs = 10
# Wall time: 1min 17s

SOM_old training started at 2019-07-04 18:21:47.579465
End of epoch 1
End of epoch 2
End of epoch 3
End of epoch 4
End of epoch 5
End of epoch 6
End of epoch 7
End of epoch 8
End of epoch 9
End of epoch 10
End of epoch 11
End of epoch 12
End of epoch 13
End of epoch 14
End of epoch 15
End of epoch 16
End of epoch 17
End of epoch 18
End of epoch 19
End of epoch 20
End of epoch 21
End of epoch 22
End of epoch 23
End of epoch 24
End of epoch 25
End of epoch 26
End of epoch 27
End of epoch 28
End of epoch 29
End of epoch 30
End of epoch 31
End of epoch 32
End of epoch 33
End of epoch 34
End of epoch 35
End of epoch 36
End of epoch 37
End of epoch 38
End of epoch 39
End of epoch 40
End of epoch 41
End of epoch 42
End of epoch 43
End of epoch 44
End of epoch 45
End of epoch 46
End of epoch 47
End of epoch 48
End of epoch 49
End of epoch 50
End of epoch 51
End of epoch 52
End of epoch 53
End of epoch 54
End of epoch 55
End of epoch 56
End of epoch 57
End of epoch 58
End of epoch 59
End of epo

In [11]:
print(
    som_new.ssdHist[-1]/som_old.ssdHist[-1]
)

0.9289391790328894


In [12]:
print("SOM_old:")
som_old.plotSSD()
print("SOM_new:")
som_new.plotSSD()

SOM_old:


SOM_new:


## Performance comparison with old SOM:

In [18]:
%%time
import time
import datetime
from math import ceil

# choosing and preparing dataset
dataset_name='pk'
data = datasets[dataset_name]
N = len(data['features'].index) # number of datapoints
l = ceil((5*N**.5)**.5) # tamanho do lado da grid quadrada de neurônios
X = data['features'].values.copy()
X1, X2 = scale_feat(X,X,scaleType='min-max')
X=X1

n = 100

params = {
     "X":         X
    ,"alpha0":    0.1
    ,"sigma0":    3 
    ,"nEpochs":   5
    ,"verboses":  0
    #,"saveNeuronsHist": False
}

som_old = SOM_2D(l, l, X.shape[1]); som_old.init(X)
som_new = SOM(l, l)

print("n={}".format(n))

t0 = time.time()
for i in range(n): som_old.train(**params, batchSize=len(X)); 
t1 = time.time()
total_old = (t1-t0)/n
print("som_old finished at {}".format(datetime.datetime.now()))

t0 = time.time()
for i in range(n): som_new.fit(**params)
t1 = time.time()
total_new = (t1-t0)/n
print("som_new finished at {}".format(datetime.datetime.now()))

print("\nOld SOM time: {}\nNew SOM time: {}\n".format(total_old,total_new))
print("New SOM {}x faster.\n".format(total_old/total_new))

n=100
som_old finished at 2019-07-04 20:28:42.488451
som_new finished at 2019-07-04 20:28:49.218428

Old SOM time: 1.6339880013465882
New SOM time: 0.0672989273071289

New SOM 24.27955491607816x faster.

CPU times: user 2min 50s, sys: 33.2 ms, total: 2min 50s
Wall time: 2min 50s


In [19]:
is_over()